In [6]:
def organise_CSV(name_network,train_number):
    loss_DF = pd.read_csv("CSV/CSV_loss_" + name_network + str(train_number) + ".csv")
    loss_DF = loss_DF.groupby(['Set','Epoch'])['Value'].mean().reset_index()
    loss_DF.to_csv("CSV/CSV_loss_" + name_network + str(train_number) + ".csv",index = False)
    
    loss_DF = pd.read_csv("CSV/CSV_confMat_" + name_network + str(train_number) + ".csv")
    loss_DF = loss_DF.groupby(['Target','Prediction','Epoch','Set'])['Value'].sum().reset_index()
    loss_DF.to_csv("CSV/CSV_confMat_" + name_network + str(train_number) + ".csv",index = False)

In [2]:
def plot_loss(name_network,train_number):
    loss_DF = pd.read_csv("CSV/CSV_loss_" + name_network + str(train_number) + ".csv")
    my_plot_loss = ggplot(aes(x="Epoch", y="Value",color = "Set"),data = loss_DF) + geom_line() 
    print(my_plot_loss)

In [8]:
def plot_IuO(name_network,train_number,epoch):
    conf_DF = pd.read_csv("CSV/CSV_confMat_" + name_network + str(train_number) + ".csv")
    a = conf_DF[conf_DF['Prediction'].apply(lambda x: x=="class0")]
    my_plot_conf_mat = ggplot(aes(x = "Epoch", y = "Value", color = "Target",size = "Set"),data = a) + geom_line()
    print(my_plot_conf_mat)

In [ ]:
def plot_mat_confusion(name_network,train_number,epoch):
    conf_DF = pd.read_csv("CSV/CSV_confMat_" + name_network + str(train_number) + ".csv")
    loss_DF = pd.read_csv("CSV/CSV_confMat_" + name_network + str(train_number) + ".csv")

    loss_DF = loss_DF[loss_DF["Epoch"]==1]
    loss_DF = loss_DF[loss_DF["Set"]=="train"]

    conf_mat_for_plot = np.zeros((20,20))
    for i in range(20):
        for j in range(20):
            conf_mat_for_plot[i,j] = loss_DF.loc[(loss_DF["Prediction"]==("class"+str(i))) & (loss_DF["Target"]==("class"+str(j)))].iloc[:,4].values

    fig, ax = plt.subplots()
    fig.set_size_inches(20, 20)
    sns.heatmap(conf_mat_for_plot,annot=True,ax=ax)
    plt.show()

In [1]:
def plot_training(loss_train,conf_mat_train,IoU_classes_train,IoU_total_train):
    fig = plt.figure(1)
    plt.clf()
    ax = fig.add_subplot(111)
    ax.set_aspect(1)
    res = ax.imshow(np.array(conf_mat_train[-1]), cmap=plt.cm.jet, 
                    interpolation='nearest')
    cb = fig.colorbar(res)
    plt.show()
    
    ggplot(aes(),data = IoU_classes_train)+ geom_point(x="0",y="1")
    

    



#plot_training(loss_train,conf_mat_train,IoU_classes_train,IoU_total_train)